https://www.youtube.com/watch?v=8PnxJ3s3Cwo&list=PLHOsBEAyYj3xf4i20sCA5o8MgVW5sIiHD&index=11   
위 유튜브 기준 + Docs 참조한 내용 정리.

In [1]:
import torch
import torchvision
import torchvision.transforms as tr
from torch.utils.data import DataLoader, Dataset
import numpy as np

#### torchvision.transforms란?   
데이터를 불러오면서 바로 전처리할 수 있게 해주는 라이브러리  
   
#### DataLoader란?   
데이터를 배치사이즈 형태로 만들어서, 실제로 학습할 때 이용할 수 있도록 만드는 라이브러리   
**Docs: Dataset을 샘플에 쉽게 접근할 수 있게 순회 가능한 객체(iterable)로 감쌈.**
#### Dataset란?
튜닝할 때 사용할 껀데, 차후 말씀드림.   
**Docs: 샘플과 정답(label)을 저장**

# 1. 파이토치 제공 데이터 사용

#### tf.Compose란?
안에있는 내용 1열로 순차적 수행.   
Compose안에는 PIL이미지만 들어갈 수 있다.(https://velog.io/@olxtar/Torchvision-PIL-torch.Tensor-PIL-Image <= PIL,numpy,tensor 이미지 정리)

In [2]:
transf = tr.Compose([tr.Resize(8), tr.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
trainset[0][0].size()
#torch.Size([3, 8, 8]) = Channel, Hight, Width
#일반적인 이미지 = 8,8,3 = H,W,C

torch.Size([3, 8, 8])

In [4]:
trainset[0][1].size()

AttributeError: 'int' object has no attribute 'size'

### DataLoader란?
Dataset을 batch 기반의 딥러닝 모델 학습을 위해 배치형태로 만들어서 우리가 실제로 학습할 때 이용할 수 있게 형태를 만들어 주는 기능

#### num_workers란?
학습 도중 CPU의 작업을 몇 개의 코어를 사용해서 진행할 지에 대한 설정 파라미터.    
https://jybaek.tistory.com/799

In [8]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=50, shuffle=True, num_workers=2)

In [10]:
len(trainloader) # CIFAR가 50000개인데, batch를 50으로 했으니까 50짜리 batch 1000개 나오는거다.

1000

trainloader안에 실제 값을 보고싶다면?
##### iter, next
한 묶음씩 불러오겠다는 뜻.

In [11]:
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [12]:
images.size() #[50, 3, 8, 8] = [batch_size, channel, hight, width]

torch.Size([50, 3, 8, 8])

# 2. 같은 클래스 별 폴더 이미지 데이터 이용
우리가 정리를 잘해놔서 폴더안에 클래스별로 나눠논 상황이라면?   
*'./class/tiger'*   
*'./class/lion'*     
이런식으로   
   
   
#### torchvision.datasets.ImageFolder <-이걸써라
*'./class'* 안에 있는 이미지들을 알아서 Search해주고,   
각각의 폴더에 대해서 labeling을 알아서 해줌!!! 엄청 좋죠?   
추가적으로 *transform=transf*을 통해서 전처리를 해줄 수 있다는것.   
정리하자면,   
*trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)*   
이 한줄로 **데이터 불러오기 + 라벨링 + 전처리 ** 쌉possible


In [13]:
transf = tr.Compose([tr.Resize(16), tr.ToTensor()])
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf)
trainloader = DataLoader(trainset, batch_size=1, shuffle=True, num_workers=2)
print(len(trainloader))
#파일없어서 에러남

FileNotFoundError: [Errno 2] No such file or directory: './class'

# 3. (개인데이터)직접 데이터 불러오기 (2가지 방법)
### 3-1 첫 번째 방법은 딥러닝호형이 선호하는 방법   

In [14]:
'''
넘파이 형태로 들어왔다고 가정,
32x32, 3Ch짜리 이미지가 20개 있다고 가정
그에 따른 레이블 20개 있다고 가정
'''

train_images = np.random.randint(256, size=(20, 32, 32,3))
train_labels = np.random.randint(2, size=(20, 1))

print(train_images.shape, train_labels.shape)

(20, 32, 32, 3) (20, 1)


In [15]:
# 수동적인 데이터를 불러오려면, 아래 ***코드 양식***을 기억해라!!!!!@!#@!#!@#
# init, getitem, len함수는 정해져있고 세부 내용은 변경하면 된다고 생각해라.

class TensorData(Dataset):
    
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.x_data = self.x_data.permute(0, 3, 1, 2) # permute함수를 이용해서 순서를 바꿔주도록 한 것임.
                                                      # 이미지 개수, 채널 수, 이미지 높이, 너비
        self.y_data = torch.LongTensor(y_data)
        self.len = self.y_data.shape[0]
    
    def __getitem__(self, index):
        
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len


In [16]:
train_data = TensorData(train_images, train_labels)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)


In [17]:
train_data[0][0].size()

torch.Size([3, 32, 32])

In [18]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

In [19]:
images.size()

torch.Size([10, 3, 32, 32])

### [번외] 호형이 torchvision.datasets.ImageFolder를 쓰지 않는 개인적인 이유
- 폴더 정리 및 변경을 하지 못하는 경우
    - 다른 작업과 공용으로 사용
    - 폴더가 아닌 SQL 같은 곳에서 넘어오는 경우   


- 파이토치에서 제공하는 transform 종류가 제한적
    - OpenCV가 훨씬 더 많음
    - 직접 이미지 전처리에 대한 함수를 만들어야 되는 경우도 있음.
    - 이러한 디테일한 이유 때문에 파이토치에서 제공하는 전처리 작업은 사용 안함.
    
***그래서, 호형은 데이터 전처리 "모듈"을 따로 만들어놓고 사용한다네***   
ex.   
import preprocessing   

train_images, train_labels = preprocessing(train_images, train_labels)


## 파이토치로 무조건 전처리하기(고급ver)
https://www.youtube.com/watch?v=8PnxJ3s3Cwo&list=PLHOsBEAyYj3xf4i20sCA5o8MgVW5sIiHD&index=11   
16분정도

In [ ]:
'''
개인 데이터에다가 transform 이용하기.
'''

In [24]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform=None):
        
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.len

class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2, 0, 1)
        return inputs, torch.LongTensor(labels)

class LinearTensor:
    
    def __init__(self, slope=1, bias=0):
        self.slope = slope
        self.bias = bias
    
    def __call__(self, sample):
        inputs, labels = sample
        inputs = self.slope*inputs + self.bias
        return inputs, labels
    

In [29]:
trans = tr.Compose([ToTensor(), LinearTensor(2, 5)])
ds1 = MyDataset(train_images, train_labels, transform=trans)
train_loader1 = DataLoader(ds1, batch_size=10, shuffle=True)


In [30]:
first_data = ds1[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [31]:
dataiter1 = iter(train_loader1)
images1, labels1 = dataiter1.next()

In [32]:
images1

tensor([[[[ 75., 305., 491.,  ..., 415., 311., 115.],
          [227., 235., 351.,  ..., 401., 161.,   5.],
          [111., 441., 451.,  ..., 449.,   9.,  99.],
          ...,
          [445., 123.,  23.,  ..., 333., 403., 271.],
          [273., 213., 147.,  ..., 305., 169., 367.],
          [ 43., 195., 491.,  ..., 279.,  95.,  79.]],

         [[391.,  31., 445.,  ..., 269., 185., 195.],
          [497., 477., 411.,  ..., 353., 297., 103.],
          [321., 289., 373.,  ..., 441., 361.,  51.],
          ...,
          [321., 211.,  73.,  ..., 355., 187., 263.],
          [465., 217.,  79.,  ..., 273., 229., 233.],
          [233., 377., 455.,  ..., 265., 133.,  63.]],

         [[ 31., 337., 515.,  ..., 197., 241., 265.],
          [155., 175., 445.,  ..., 243.,  39., 349.],
          [201., 511., 235.,  ..., 295., 345., 391.],
          ...,
          [131., 471., 265.,  ..., 203., 127., 131.],
          [447., 317., 441.,  ...,  49.,  11., 461.],
          [ 57., 139.,  45.,  ...

## 개인 데이터 + torchvision.transforms(고급)
tr.compose이용해서 하고싶을경우(class재정의하는게 너무 힘들어요!!~~일경우)

In [33]:

class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform=None):
        
        self.x_data = x_data
        self.y_data = y_data
        self.transform = transform
        self.len = len(y_data)
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample
    
    def __len__(self):
        return self.len
    
    

    
class MyTransform:
    
    def __call__(self, sample):        
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2, 0, 1)
        labels = torch.FloatTensor(labels)
        
        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128),tr.ToTensor(), tr.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        final_output = transf(inputs)
        
        return final_output, labels
        

In [35]:
ds2 = MyDataset(train_images, train_labels, transform=MyTransform())
train_loader2 = DataLoader(ds2, batch_size=10, shuffle=True)

In [36]:
first_data = ds2[0]
features, labels = first_data
print(type(features), type(labels))

<class 'torch.Tensor'> <class 'torch.Tensor'>


In [37]:
dataiter2 = iter(train_loader2)
images2 , labels2 = dataiter2.next()

In [38]:
images2

tensor([[[[ 0.3569,  0.3569,  0.2314,  ...,  0.1922,  0.2000,  0.2000],
          [ 0.3569,  0.3569,  0.2314,  ...,  0.1922,  0.2000,  0.2000],
          [ 0.2941,  0.2941,  0.1765,  ...,  0.0588,  0.0667,  0.0667],
          ...,
          [ 0.7569,  0.7569,  0.5843,  ...,  0.2157,  0.3333,  0.3333],
          [ 0.8118,  0.8118,  0.6157,  ...,  0.2157,  0.3333,  0.3333],
          [ 0.8118,  0.8118,  0.6157,  ...,  0.2157,  0.3333,  0.3333]],

         [[ 0.8431,  0.8431,  0.7412,  ..., -0.2784, -0.3020, -0.3020],
          [ 0.8431,  0.8431,  0.7412,  ..., -0.2784, -0.3020, -0.3020],
          [ 0.8118,  0.8118,  0.7098,  ..., -0.2314, -0.2706, -0.2706],
          ...,
          [-0.2549, -0.2549, -0.2863,  ..., -0.2471, -0.3020, -0.3020],
          [-0.1608, -0.1608, -0.2000,  ..., -0.3961, -0.4745, -0.4745],
          [-0.1608, -0.1608, -0.2000,  ..., -0.3961, -0.4745, -0.4745]],

         [[ 0.4667,  0.4667,  0.3490,  ...,  0.5059,  0.5137,  0.5137],
          [ 0.4667,  0.4667,  